# Simulation of a community as it can be today
## No battery export!
## No probabilities included

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import paper_classes_2 as pc
import Post_processing as pp
import itertools
import pickle
from termcolor import colored
import threading
import csv
import os

In [2]:
def save_results(agg_results,filename):
    '''
    Description
    -----------
    Save the results of a dataframe in a csv file using filename
    
    Parameters
    -----------
    agg_results (pd.DataFrame): aggregated results
    filename (str): name of the file
    Returns
    ------
    None
    '''
    global_lock = threading.Lock()
    while global_lock.locked():
        continue
    global_lock.acquire()

    with open(filename, 'a', newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(agg_results.values.flatten())
    global_lock.release()

In [3]:
def plot_dispatch_orig(pv, demand, E, week=30):
    """ 
    Description
    -----------
    Visualize dispatch algorithm for a specific week
    Parameters
    -----------
    demand (pd.Series): demand production
    pv (pd.Series): pv production
    E (dict):  Energy flows. Dictionary of pd.Series: res_pv, grid2load, store2inv, LevelOfCharge
    Returns
    -----------
    None
    """

    sliced_index = (pv.index.week==week)
    pv_sliced = pv[sliced_index]
    demand_sliced = demand[sliced_index]
    self_consumption = E['inv2load'][sliced_index]
    
    direct_self_consumption = np.minimum(pv_sliced,demand_sliced)# E['inv2load'][sliced_index]
    indirect_self_consumption = self_consumption-direct_self_consumption
    res_pv_sliced = E['res_pv'][sliced_index]
    grid2load_sliced = E['grid2load'][sliced_index]
    store2inv_sliced = E['store2inv'][sliced_index]
    LevelOfCharge = E['LevelOfCharge'][sliced_index]
    inv2grid = E['inv2grid'][sliced_index]
    grid2load = E['grid2load'][sliced_index]
    aux=np.maximum(0,self_consumption)

    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(17, 4*3), frameon=False,
                             gridspec_kw={'height_ratios': [3, 1, 1], 'hspace': 0.04})

    #fig, ax = plt.subplots(figsize=(17, 4))
    axes[0].plot(demand_sliced.index, demand_sliced, color='black', lw=2,label='demand')
    axes[0].plot(pv_sliced.index, pv_sliced, color='black',ls='--', lw=2,label='PV')
    axes[0].fill_between(direct_self_consumption.index, 0, direct_self_consumption, color='orange', alpha=.8, label='DSC')
    axes[0].fill_between(pv_sliced.index, self_consumption, pv_sliced , where=pv_sliced<demand_sliced,color='blue', hatch='//',
                         alpha=.3,label='ISC')
    axes[0].fill_between(pv_sliced.index, direct_self_consumption, pv_sliced ,where=pv_sliced>demand_sliced, color='gold', alpha=.3,label='Excess PV')

    axes[0].fill_between(grid2load_sliced.index,self_consumption,demand_sliced,color='red',alpha=.2, label='grid2load')
        

    #axes[0].plot(grid2load_sliced.index, grid2load_sliced, color='red', ls=":", lw=1)
    axes[0].set_ylim([0, axes[0].get_ylim()[1] ])
    axes[0].set_ylabel('Power (kW)')

    axes[1].fill_between(LevelOfCharge.index, 0, LevelOfCharge, color='grey', alpha=.2, label='SOC')
    axes[1].set_ylabel('State of Charge (kWh)')

    axes[2].fill_between(inv2grid.index, 0, inv2grid, color='green', alpha=.2,label='injected2grid')
    axes[2].fill_between(inv2grid.index, 0, -grid2load, color='red', alpha=.2,label='grid drawn')
    axes[2].set_ylabel('In/out from grid (kW)')
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    return



In [4]:
def plot_dispatch(pv, demand, E,param, week=30):
    """ 
    Description
    -----------
    Visualize dispatch algorithm for a specific week
    Parameters
    -----------
    demand (pd.Series): demand production
    pv (pd.Series): pv production
    E (dict):  Energy flows. Dictionary of pd.Series: res_pv, grid2load, store2inv, LevelOfCharge
    param (dict): simulation parameters
    week (int): week number of the year to be plotted
    Returns
    -----------
    None
    """

    sliced_index = (pv.index.week==week)
    pv_sliced = pv[sliced_index]
    demand_sliced = demand[sliced_index]
    self_consumption = E['inv2load'][sliced_index]
    if 'store2grid' in E.keys():
        store2grid_sliced=E['store2grid'][sliced_index]*param['InverterEfficiency']
    direct_self_consumption = np.minimum(pv_sliced,demand_sliced)# E['inv2load'][sliced_index]
    indirect_self_consumption = self_consumption-direct_self_consumption
    res_pv_sliced = E['res_pv'][sliced_index]
    grid2load_sliced = E['grid2load'][sliced_index]
    store2inv_sliced = E['store2inv'][sliced_index]
    LevelOfCharge = E['LevelOfCharge'][sliced_index]
    inv2grid = E['inv2grid'][sliced_index]
    grid2load = E['grid2load'][sliced_index]
    aux=np.maximum(0,self_consumption)

    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(17, 4*3), frameon=False,
                             gridspec_kw={'height_ratios': [3, 1, 1], 'hspace': 0.04})

    #fig, ax = plt.subplots(figsize=(17, 4))
    axes[0].plot(demand_sliced.index, demand_sliced, color='black', lw=2,label='demand')
    axes[0].plot(pv_sliced.index, pv_sliced, color='black',ls='--', lw=2,label='PV')
    axes[0].fill_between(direct_self_consumption.index, 0, direct_self_consumption, color='orange', alpha=.8, label='DSC')
    axes[0].fill_between(pv_sliced.index, self_consumption, pv_sliced , where=pv_sliced<demand_sliced,color='blue', hatch='//',
                         alpha=.3,label='ISC')
    axes[0].fill_between(pv_sliced.index, direct_self_consumption, pv_sliced ,where=pv_sliced>demand_sliced, color='gold', alpha=.3,label='Excess PV')

    axes[0].fill_between(grid2load_sliced.index,self_consumption,demand_sliced,color='red',alpha=.2, label='grid2load')

    #axes[0].plot(grid2load_sliced.index, grid2load_sliced, color='red', ls=":", lw=1)
    axes[0].set_ylim([0, axes[0].get_ylim()[1] ])
    axes[0].set_ylabel('Power (kW)')

    axes[1].fill_between(LevelOfCharge.index, 0, LevelOfCharge, color='grey', alpha=.2, label='SOC')
    axes[1].set_ylabel('State of Charge (kWh)')
    if 'store2grid' in E.keys():
        axes[2].fill_between(store2grid_sliced.index, 0, store2grid_sliced, color='blue', alpha=.2,label='store2grid')

        axes[2].fill_between(inv2grid.index, store2grid_sliced, inv2grid, color='green', alpha=0.2,label='PV2grid')
    else:
        axes[2].fill_between(inv2grid.index, 0, inv2grid, color='green', alpha=0.2,label='PV2grid')
    axes[2].fill_between(inv2grid.index, 0, -grid2load, color='red', alpha=.2,label='grid drawn')
    axes[2].set_ylabel('In/out from grid (kW)')
    axes[0].legend()
    axes[1].legend()
    axes[2].legend()
    return



In [5]:
def print_analysis_orig(pv, demand, param, E):
    """ 
    Description
    -----------
    Print statistics and information of the dispatched solution
    Parameters
    -----------
    pv (pd.Series): PV timeseries
    demand (pd.Series): demand timeseries
    param (dict): dictionary of technical parameters
    E (dict): dictionary of energy flows as estimated by the algorithm
    Returns
    -----------
    None
    """
    timestep = param['timestep']
    SelfConsumption = np.sum(E['inv2load']) * timestep
    TotalFromGrid = np.sum(E['grid2load']) * timestep
    TotalToGrid = np.sum(E['inv2grid']) * timestep
    TotalLoad = demand.sum() * timestep
    TotalPV = pv.sum() * timestep
    TotalBatteryGeneration = np.sum(E['store2inv']) * timestep
    TotalBatteryConsumption = np.sum(E['pv2store']) * timestep
    BatteryLosses=E['batt_losses'].sum()
    InverterLosses=E['inv_losses'].sum()
    SelfConsumptionRate = SelfConsumption / TotalPV * 100             # in %
    SelfSufficiencyRate = SelfConsumption / TotalLoad * 100
    AverageDepth = TotalBatteryGeneration / (365 * param['BatteryCapacity'])
    
    Nfullcycles = 365 * AverageDepth
    residue = TotalPV + TotalFromGrid - TotalToGrid - BatteryLosses - InverterLosses - TotalLoad-E['LevelOfCharge'][-1]*( param['InverterEfficiency'])

    print ('Total yearly consumption: {:1g} kWh'.format(TotalLoad))
    print ('Total PV production: {:1g} kWh'.format(TotalPV))
    print ('Self Consumption: {:1g} kWh'.format(SelfConsumption))
    print (colored('Total fed to the grid: {:1g} kWh'.format(TotalToGrid),'red'))
    print ('Total bought from the grid: {:1g} kWh'.format(TotalFromGrid))
    print (colored('Self consumption rate (SCR): {:1g}%'.format(SelfConsumptionRate),'red'))
    print (colored('Self sufficiency rate (SSR): {:1g}%'.format(SelfSufficiencyRate),'red'))
    print ('Amount of energy provided by the battery: {:1g} kWh'.format(TotalBatteryGeneration))
    print ('Average Charging/Discharging depth: {:1g}'.format(AverageDepth))
    print ('Number of equivalent full cycles per year: {:1g} '.format(Nfullcycles))
    print ('Total battery losses: {:1g} kWh'.format(BatteryLosses))
    print ('Total inverter losses: {:1g} kWh'.format(InverterLosses))
    print ('Residue (check): {:1g} kWh'.format(residue))

In [6]:

def print_analysis(pv, demand, param, E,save):
    """ 
    Description
    -----------
    Print statistics and information of the dispatched solution
    
    Modified by alefunxo (01.07.2019) to include other parameters and forbid charge and discharge in the same timeslot
    and inverter and battery losses.
    
    Parameters
    -----------
        pv (pd.Series): PV timeseries
        demand (pd.Series): demand timeseries
        param (dict): dictionary of technical parameters
        E (dict): dictionary of energy flows as estimated by the algorithm
        save (boolean): save in a csv file the aggregated results
    Returns
    -----------
    None
    """
    print('\n%%%%%%%%%%%%%%%%%')
    print('Printing analysis')
    timestep = param['timestep']
    
    TotalFromGrid = np.sum(E['grid2load']) * timestep

    SelfConsumption = np.sum(E['inv2load']) * timestep
    TotalToGrid=(np.sum(E['inv2grid']))*timestep
    TotalLoad = demand.sum() * timestep
    TotalPV = pv.sum() * timestep
    TotalBatteryGeneration = np.sum(E['store2inv']) * timestep
    TotalBatteryConsumption = np.sum(E['pv2store']) * timestep
    BatteryLosses=E['batt_losses'].sum()
    InverterLosses=E['inv_losses'].sum()
    SelfConsumptionRate = SelfConsumption / TotalPV * 100             # in %
    SelfSufficiencyRate = SelfConsumption / TotalLoad * 100
    AverageDepth = TotalBatteryGeneration / (365 * param['BatteryCapacity'])
    Nfullcycles = 365 * AverageDepth
    if isinstance(param['df_prices'],pd.Series):
        bill=(E['grid2load']*param['df_prices']-E['inv2grid']*param['export_prices']).sum()
    else:
        bill=(E['grid2load']*param['df_prices'].iloc[:,0]-E['inv2grid']*param['export_prices']).sum()
    
    if 'store2grid' in E.keys():
        print('inside st2gr')
        IndirectSelfConsumption=np.sum(E['store2inv']-E['store2grid']) *param['InverterEfficiency']* timestep
        DirectSelfConsumption= np.sum(E['inv2load']) * timestep-IndirectSelfConsumption
        TotalToGridFromPV = (np.sum(E['inv2grid'])-np.sum(E['store2grid'])*param['InverterEfficiency']) * timestep
        TotalToGridFromBatt = (np.sum(E['inv2grid']) * timestep-TotalToGridFromPV)
        store2grid=np.sum(E['store2grid'])*timestep
        flag_sp_p2p=E['flag_sp_p2p'].sum()
        flag_no_sp_p2p=E['flag_no_sp_p2p'].sum()
    else:
        flag_sp_p2p=0
        flag_no_sp_p2p=0

    residue = TotalPV + TotalFromGrid - TotalToGrid - BatteryLosses - InverterLosses - TotalLoad-E['LevelOfCharge'][-1]*( param['InverterEfficiency'])


    print ('Total yearly consumption: {:1g} kWh'.format(TotalLoad))
    print ('Total PV production: {:1g} kWh'.format(TotalPV))
    print ('Self Consumption: {:1g} kWh'.format(SelfConsumption))
    if 'store2grid' in E.keys():
        print ('Direct Self Consumption: {:1g} kWh'.format(DirectSelfConsumption))
        print ('Indirect Self Consumption: {:1g} kWh'.format(IndirectSelfConsumption))
        print ('Total fed to the grid from PV: {:1g} kWh'.format(TotalToGridFromPV))
        print ('Total fed to the grid from battery: {:1g} kWh'.format(TotalToGridFromBatt))
        print ('The amount of times energy when surplus was exchanged was: {}'.format(flag_sp_p2p))
        print ('The amount of times energy was exchanged when no surplus was: {}'.format(flag_no_sp_p2p))
        
    print (colored('Total fed to the grid: {:1g} kWh'.format(TotalToGrid),'red'))
    print ('Total bought from the grid: {:1g} kWh'.format(TotalFromGrid))
    print (colored('Self consumption rate (SCR): {:1g}%'.format(SelfConsumptionRate),'red'))
    print (colored('Self sufficiency rate (SSR): {:1g}%'.format(SelfSufficiencyRate),'red'))
    print ('Self consumption rate (SCR): {:.3g}%'.format(SelfConsumptionRate))
    print ('Self sufficiency rate (SSR): {:.3g}%'.format(SelfSufficiencyRate))
    print ('Amount of energy provided by the battery: {:1g} kWh'.format(TotalBatteryGeneration))
    print ('Average Charging/Discharging depth: {:1g}'.format(AverageDepth))
    print ('Number of equivalent full cycles per year: {:1g} '.format(Nfullcycles))
    print ('Total battery losses: {:1g} kWh'.format(BatteryLosses))
    print ('Total inverter losses: {:1g} kWh'.format(InverterLosses))
    print ('Total bill: {:1g} USD/a '.format(bill))
    print ('Residue (check): {:1g} kWh'.format(residue))
    print('%%%%%%%%%%%%%%%%%')
    print(save)
    np.random.seed()
    id_rep=np.random.randint(1, 100000, 1)[0]#to get the different responses in the loops
    if save:
        
        print('%%%%%%%%%%%%%%%%%')
        print('Saving analysis')
        
        if 'store2grid' in E.keys():
            
            filename='Switzerland/Output/aggregated_results_p2p.csv'
            agg_results=pd.DataFrame([param['pv_penetration'],param['batt_penetration'],id_rep,TotalFromGrid,IndirectSelfConsumption,
                      DirectSelfConsumption,TotalToGridFromPV,TotalToGridFromBatt,
                      SelfConsumption,TotalToGrid,TotalLoad,TotalPV,TotalBatteryGeneration,TotalBatteryConsumption,
                      BatteryLosses,InverterLosses,SelfConsumptionRate,SelfSufficiencyRate,AverageDepth,Nfullcycles,bill,residue,
                      E['flag_no_sp_p2p'].sum(),E['flag_sp_p2p'].sum(),
                      param['use_cluster1_probabilities'],param['kW_dis']])
            save_results(agg_results,filename)
        else:
            filename='Switzerland/Output/aggregated_results_as_today.csv'
            agg_results=pd.DataFrame([param['pv_penetration'],param['batt_penetration'],id_rep,TotalFromGrid,
                      SelfConsumption,TotalToGrid,TotalLoad,TotalPV,TotalBatteryGeneration,TotalBatteryConsumption,
                      BatteryLosses,InverterLosses,SelfConsumptionRate,SelfSufficiencyRate,AverageDepth,Nfullcycles,bill,residue,
                      param['use_cluster1_probabilities'],param['kW_dis']])
            save_results(agg_results,filename)
    return_values=True
    if return_values:

        return pd.DataFrame([[param['pv_penetration'],param['batt_penetration'],id_rep,TotalFromGrid,
                      SelfConsumption,TotalToGrid,TotalLoad,TotalPV,TotalBatteryGeneration,TotalBatteryConsumption,
                      BatteryLosses,InverterLosses,SelfConsumptionRate,SelfSufficiencyRate,AverageDepth,Nfullcycles,bill,residue,
                      param['use_cluster1_probabilities'],param['kW_dis'],flag_no_sp_p2p,flag_sp_p2p]],columns=['pv_penetration','batt_penetration',
                      'id_rep','TotalFromGrid','SelfConsumption','TotalToGrid','TotalLoad','TotalPV','TotalBatteryGeneration',
                      'TotalBatteryConsumption','BatteryLosses','InverterLosses','SelfConsumptionRate','SelfSufficiencyRate',
                      'AverageDepth','Nfullcycles','bill','residue',
                      'use_cluster1_probabilities','kW_dis','flag_no_sp_p2p','flag_sp_p2p'])



In [7]:

def dispatch_max_sc(pv, demand, param, return_series=False):
    """ 
    Description
    -----------
    Self consumption maximization pv + battery dispatch algorithm.
    The dispatch of the storage capacity is performed in such a way to maximize self-consumption:
    the battery is charged when the PV power is higher than the load and as long as it is not fully charged.
    It is discharged as soon as the PV power is lower than the load and as long as it is not fully discharged.
    
    Modified by alefunxo (01.07.2019) to include other parameters and forbid charge and discharge in the same timeslot
    and inverter and battery losses.
    
    Parameters
    -----------
    pv (pd.Series): Vector of PV generation, in kW DC (i.e. before the inverter)
    demand (pd.Series): Vector of household consumption, kW
    param (dict): Dictionary with the simulation parameters:
            timestep (float): Simulation time step (in hours)
            BatteryCapacity: Available battery capacity (i.e. only the the available DOD), kWh
            BatteryEfficiency: Battery round-trip efficiency, -
            InverterEfficiency: Inverter efficiency, -
            MaxPower: Maximum battery charging or discharging powers (assumed to be equal), kW
    return_series(bool): if True then the return will be a dictionary of series. Otherwise it will be a dictionary of ndarrays.
                    It is reccommended to return ndarrays if speed is an issue (e.g. for batch runs).
    Returns
    -----------
    dict: Dictionary of Time series
    """

    bat_size_e_adj = param['BatteryCapacity']
    bat_size_p_adj = param['MaxPower']
    n_bat = param['BatteryEfficiency']
    n_inv = param['InverterEfficiency']
    timestep = param['timestep']
    # We work with np.ndarrays as they are much faster than pd.Series
    Nsteps = len(pv)
    LevelOfCharge = np.zeros(Nsteps)
    pv2store = np.zeros(Nsteps)
    inv2grid = np.zeros(Nsteps)
    store2inv = np.zeros(Nsteps)
    #store2load = np.zeros(Nsteps)
    grid2store = np.zeros(Nsteps) # TODO Always zero for now.
    #store2grid = np.zeros(Nsteps)
    batt_losses = np.zeros(Nsteps)
    #prices=np.ones(Nsteps)*.07
    #Load served by PV
    pv2inv = np.minimum(pv, demand / n_inv)  # DC direct self-consumption

    #Residual load
    res_load = (demand - pv2inv * n_inv)  # AC
    inv2load = pv2inv * n_inv  # AC

    #Excess PV
    res_pv = np.maximum(pv - demand/n_inv, 0)  # DC
    #PV to storage after eff losses
    pv2inv = pv2inv.values

    #first timestep = 0
    LevelOfCharge[0] = 0  # bat_size_e_adj / 2  # DC
    
    for i in range(1,Nsteps):
        #PV to storage
        if LevelOfCharge[i-1] >= bat_size_e_adj:  # if battery is full
                pv2store[i] = 0
        else: #if battery is not full
            if LevelOfCharge[i-1] + res_pv[i] * timestep > bat_size_e_adj:  # if battery will be full after putting excess
                pv2store[i] = min((bat_size_e_adj - LevelOfCharge[i-1]) / timestep, bat_size_p_adj)
            else:
                pv2store[i] = min(res_pv[i], bat_size_p_adj)

        #Storage to load 
        ######################################
        batt_losses[i]=pv2store[i]*(1-n_bat)
        if pv2store[i]==0:#not charging
            store2inv[i] = min(bat_size_p_adj,  # DC
                           res_load[i] / n_inv,
                           LevelOfCharge[i-1] / timestep)
            
            
        ######################################a
        
        #SOC
        LevelOfCharge[i] = min(LevelOfCharge[i-1] - batt_losses[i]-(store2inv[i] - pv2store[i] - grid2store[i]) * timestep,  # DC
                               bat_size_e_adj)
    
    pv2inv = pv2inv + res_pv - pv2store
    inv2load = inv2load + store2inv * n_inv  # AC
    inv2grid = (res_pv - pv2store) * n_inv  # AC
    grid2load = demand - inv2load  # AC
    inv_losses=(pv2inv+store2inv)*(1-n_inv)
    #MaxDischarge = np.minimum(LevelOfCharge[i-1]*BatteryEfficiency/timestep,MaxPower)


    #Potential Grid to storage  # TODO: not an option for now in this strategy
    # GridPurchase = False

    out = {'pv2inv': pv2inv,
            'res_pv': res_pv,
            'pv2store': pv2store,
            'inv2load': inv2load,
            'grid2load': grid2load,
            'store2inv': store2inv,
            'LevelOfCharge': LevelOfCharge,
            'inv2grid': inv2grid,
            'batt_losses':batt_losses,
            'inv_losses':inv_losses
            # 'grid2store': grid2store
            }
    if not return_series:
        out_pd = {}
        for k, v in out.items():  # Create dictionary of pandas series with same index as the input pv
            out_pd[k] = pd.Series(v, index=pv.index)
        out = out_pd
    return out

In [8]:
def flag_selection(df,list_product,community_size,list_pv_penetration,list_batt_penetration):
    '''
    Description
    -----------
    Put a flag (boolean True) where there is PV or Battery randomly 
    Parameters
    -----------
    df (pd.DataFrame): dataframe including the PV_size and the name of every household in the community
    list_product (list): list of combinations of PV and battery penetration
    community_size (int): size of the community
    list_pv_penetration (list): list of PV penetration
    list_batt_penetration (list): list of battery penetration
    Returns
    -----------
    df (pd.DataFrame): dataframe with one boolean column for every combination of PV and battery penetration,
                            indicating whether the household use PV and/or battery or not.
    '''
    list_names=['sub_'+str(i[0])+'_'+str(i[1]) for i in list_product]
    for i in list_names:
        df[i]=False
    dict_pv_df={}
    dict_batt_df={}
    j=0
    l=0
    for i in list_pv_penetration: 
        #print('-----')
        dict_pv_df[j]=np.random.choice(df.index,int(len(df)*(i/100)), replace=False)
        for k in list_batt_penetration:
            #print(int(len(dict_pv_df[j])*(k/100)))
            dict_batt_df[l]=np.random.choice(dict_pv_df[j],int(len(dict_pv_df[j])*(k/100)), replace=False)
            df.iloc[dict_batt_df[l],l+2]=True
            l+=1
        j+=1
    return df

In [9]:
def save_obj(obj, name ):
    '''
    Description
    -----------
    saves an object in Switzerland/Output/ folder with name "name" as a pickle object
    
    Parameters
    -----------
    obj (whatever): Object to be saved (array, list, dict, DataFrame...)
    name (str): name with which the object will be saved 
    Returns
    -----------
    None
    '''
    with open('Switzerland/Output/'+name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    '''
    Description
    -----------
    loads a pickle object from the folder Switzerland/Output/ with name "name"
    Parameters
    -----------
    name (str): object name to be loaded
    Returns
    -----------
    None
    '''
    with open('Switzerland/Output/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [10]:
def get_bill(E,param):
    '''
    Description
    -----------
    Calculates the bill using the grid2load, inv2grid, df_prices and export_prices variables
    Parameters
    -----------
    
    E (dict): Energy flows dictionary
    param (dict): simulation parameters
    Returns
    -----------
    None
    
    '''
    if isinstance(param['df_prices'],pd.Series):
        return (E['grid2load']*param['df_prices']-E['inv2grid']*param['export_prices']).sum()
    else:
        return (E['grid2load']*param['df_prices'].iloc[:,0]-E['inv2grid']*param['export_prices']).sum()

In [17]:
def core(pv_penetration,batt_penetration,df_CH,selection,list_names,param_tech,param_tech_no_batt):
    '''
    Description
    -----------
    Core function of the simulation of communities as today. Does the dispatch of electricity for every
    household taking into account whether the houses have or not PV and battery for the selected 
    PV and battery penetration parameters. Recalculates the outputs for the whole community taking
    the surplus generated inside the community (new input as PV generation for the dispatching)
    and the residual load after self-consumption (new input as demand for the dispatching).
    saves the results in pickle objects to calculate the bill after.
    Parameters
    -----------
    pv_penetration (int): % of pv penetration in the community
    batt_penetration (int): % of battery penetration in the community
    df_CH (pd.DataFrame): Dataframe including individual electricity consumption (632 houses)
    selection (pd.DataFrame): Dataframe including PV sizes, names and selection of the houses
                                with and w/o PV and/or battery according to the pv and battery
                                penetration
    list_names (list): list of names of the houses inside the community
    param_tech (dict): technological parameters of the simulation including batteries
    param_tech_no_bat (dict): technological parameters of the simulation without batteries
    Returns
    -----------
    None
    '''
    print('########################################')
    print('PV penetration {}'.format(pv_penetration))

    print('Battery penetration {}'.format(batt_penetration))
    param_tech.update({'pv_penetration':pv_penetration,'batt_penetration':batt_penetration})
    nested_out={}
    j=0
    k=0
    PV_size_comm=0
    for i in selection.index:
        #print(i, end='')
        if selection.loc[i,'sub_'+str(pv_penetration)+'_100']:#all with PV

            PV_size_comm+=selection.PV_size[i]
            if selection.loc[i,'sub_'+str(pv_penetration)+'_'+str(batt_penetration)]: #if battery
                nested_out[i]=dispatch_max_sc(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech)
                j+=1
            else: #if only PV battery=0 kWh
                nested_out[i]=dispatch_max_sc(df_CH.E_PV*selection.PV_size[i],df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)
                k+=1
        else: #No PV
            nested_out[i]=dispatch_max_sc(df_CH.E_PV*0,df_CH.loc[:,str(selection.name[i])],param_tech_no_batt)
    
    #get all the data in a single dict
    pv2inv=pd.DataFrame()
    res_pv=pd.DataFrame()
    pv2store=pd.DataFrame()
    inv2load=pd.DataFrame()
    grid2load=pd.DataFrame()
    store2inv=pd.DataFrame()
    LevelOfCharge=pd.DataFrame()
    inv2grid=pd.DataFrame()
    batt_losses=pd.DataFrame()
    inv_losses=pd.DataFrame()
    bill_individual=pd.Series()
    
    for i in nested_out.keys():
        pv2inv['pv2inv'+str(i)]=nested_out[i]['pv2inv']
        pv2store['pv2store'+str(i)]=nested_out[i]['pv2store']
        inv2load['inv2load'+str(i)]=nested_out[i]['inv2load']
        store2inv['store2inv'+str(i)]=nested_out[i]['store2inv']
        LevelOfCharge['LevelOfCharge'+str(i)]=nested_out[i]['LevelOfCharge']
        batt_losses['batt_losses'+str(i)]=nested_out[i]['batt_losses']
        inv_losses['inv_losses'+str(i)]=nested_out[i]['inv_losses']
        res_pv['res_pv'+str(i)]=nested_out[i]['res_pv']
        inv2grid['inv2grid'+str(i)]=nested_out[i]['inv2grid']
        grid2load['grid2load'+str(i)]=nested_out[i]['grid2load']
        bill_individual.at['bill'+str(i)]=get_bill(nested_out[i],param_tech)
    nested_dict={'pv2inv':pv2inv.sum(axis=1),'res_pv':res_pv.sum(axis=1),'pv2store':pv2store.sum(axis=1),
                 'inv2load':inv2load.sum(axis=1),'grid2load':grid2load.sum(axis=1),
                 'store2inv':store2inv.sum(axis=1),'LevelOfCharge':LevelOfCharge.sum(axis=1),
                 'inv2grid':inv2grid.sum(axis=1),'batt_losses':batt_losses.sum(axis=1),
                 'inv_losses':inv_losses.sum(axis=1)}
    param_tech_no_batt=param_tech.copy()
    param_tech_no_batt.update({'InverterEfficiency':1})#nested_dict['inv2grid'] is PV generation already in AC used as input for next step
    out_comm_res=dispatch_max_sc(nested_dict['inv2grid'],nested_dict['grid2load'],param_tech_no_batt)
    out_comm_final={}
    out_comm_final['pv2inv']=nested_dict['pv2inv']
    out_comm_final['pv2store']=nested_dict['pv2store']
    out_comm_final['inv2load']=(nested_dict['inv2load']+out_comm_res['inv2load'])
    out_comm_final['store2inv']=nested_dict['store2inv']
    out_comm_final['LevelOfCharge']=nested_dict['LevelOfCharge']
    out_comm_final['res_pv']=out_comm_res['res_pv']
    out_comm_final['inv2grid']=out_comm_res['inv2grid']
    out_comm_final['grid2load']=out_comm_res['grid2load']
    out_comm_final['batt_losses']=nested_dict['batt_losses']
    out_comm_final['inv_losses']=nested_dict['inv_losses']

    out_comm_final['PV_size_comm']=PV_size_comm
    out_comm_final['df']=df_CH
    out_comm_final['param_tech']=param_tech
    out_comm_final['param_tech_no_batt']=param_tech_no_batt
    out_comm_final['nested_dict']=nested_dict
    out_comm_final['pv_penetration']=pv_penetration
    out_comm_final['batt_penetration']=batt_penetration
    out_comm_final['selection']=selection
    bill_individual.at['pv_penetration']=pv_penetration
    bill_individual.at['batt_penetration']=batt_penetration
    save_obj(out_comm_final,'community_as_today'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
    #save_results(bill_individual,'bill_individual')
    #save_results(bill_individual,'aggregated_results_bill.csv')
    save_obj(bill_individual,'bill_individual'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
    print_analysis(df_CH.E_PV*PV_size_comm, df_CH.loc[:,list_names].sum(axis=1),param_tech_no_batt,out_comm_final,1)
    save_obj(inv2grid,'i2g_community_as_today'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
    save_obj(grid2load,'g2l_community_as_today'+'_'+str(pv_penetration)+'_'+str(batt_penetration))
 


In [18]:
def initialize_community(first_time):
    '''
    Description
    -----------
    Initialize the community. When it is the first time it is used, it does a random selection of the houses,
    and the PV size and save the output in 'selection.csv', otherwhise, it loads 'selection.csv'.
    Here the technological parameters of the simulation are defined ('BatteryCapacity','BatteryEfficiency','InverterEfficiency',
                  'timestep','MaxPower','df_prices', 'export_prices':,'use_cluster1_probabilities','kW_dis','path',
                  'list_batt_penetration','list_pv_penetration')
    Parameters
    -----------
    first_time (Boolean): select whether it is the first time to generate the community or not (load a pre-defined one)
    Returns
    -----------
    param_tech (dict): technological parameters of the simulation including batteries
    param_tech_no_bat (dict): technological parameters of the simulation without batteries
    df_CH (pd.DataFrame): Dataframe including individual electricity consumption (632 houses)
    selection (pd.DataFrame): Dataframe including PV sizes, names and selection of the houses
                                with and w/o PV and/or battery according to the pv and battery
                                penetration
    list_names (list): list of names of the houses inside the community
        
    '''
    community_size=100
    timestep=1 # in hours
    pv_penetration=100
    batt_penetration=100
    list_pv_penetration=[100,75,50,25]# %
    list_batt_penetration=[100,75,50,25]# %
    df_CH=pd.read_csv('../BASOPRA_IRES/Input/df_CH_Marzia3.csv', encoding='utf8', sep=',',usecols=[*range(0, 672)],
                                      engine='python',date_parser=lambda col: pd.to_datetime(col, utc=True),infer_datetime_format=True,index_col=0)
    df_CH.index=df_CH.index.tz_convert('Europe/Brussels')
    load_prices=df_CH.iloc[:,668:]
    df_CH=df_CH.iloc[:,:668]    
    path='Switzerland/Input/'
    if timestep==1:
        df_CH=df_CH.resample('1H').sum()
        load_prices=load_prices.resample('1H').mean()
    if first_time:
        max_PV=10
        
        df_pv_ch=pd.read_excel(path+'PV_beneficiaires_Swiss.xlsx')
        df_nat=df_pv_ch[(df_pv_ch['Anlage_Projekt-Bezeichnung']=='natürliche Person')]
        df_nat_pv=df_nat[(df_nat['Anlage_Energieträger']=='Photovoltaik')]
        df_nat_pv_15=df_nat_pv[df_nat_pv['Leistung [kW]']<max_PV]['Leistung [kW]'].reset_index(drop=True)
        df_names=pd.DataFrame(df_CH.loc[:,(df_CH.sum()<7500)&(df_CH.columns!='E_PV')].sample(n=community_size,axis=1).columns)
        
        selection=pd.concat([df_nat_pv_15.round(1).sample(n=community_size).reset_index(drop=True),df_names],axis=1)
        selection.columns=['PV_size','name']
        list_product=list(itertools.product(list_pv_penetration,list_batt_penetration))
        flag_selection(selection,list_product,community_size,list_pv_penetration,list_batt_penetration)
        #print(selection.sum())
        selection.to_csv('Switzerland/Input/selection.csv',index=False)
    else:#Doesnt work TODO
        selection=pd.read_csv('Switzerland/Input/selection.csv')
        #pd.read_csv('../BASOPRA_IRES/Input/df_CH_Marzia2.csv', encoding='utf8', sep=',',usecols=demand_profiles['0'],engine='python')
    list_names=[str(i) for i in selection.iloc[:,1]]



    param_tech = {'BatteryCapacity': 10,
                  'BatteryEfficiency': .91,
                  'InverterEfficiency': .94,
                  'timestep': timestep,
                  'MaxPower': 4,
                  'df_prices':load_prices.Price_flat, #flat tariff
                  'export_prices':load_prices.Export_price,
                  'use_cluster1_probabilities':False,
                  'kW_dis':0,
                  'path':path,
                  'list_batt_penetration':list_batt_penetration,
                  'list_pv_penetration':list_pv_penetration
                 }

    # Community as today (Current state in Community)
    # Select the PV and battery penetration. Here there is no battery export.
    #We use the model from Quoilin et al. [1] for PV and battery dispatch and adapt it for the community case.
    #[1] Quoilin, S. et al., 'Quantifying self-consumption linked to solar home battery systems: Statistical analysis and economic assessment', Applied Energy, Elsevier, 2016, 182, pp. 58-67 (https://github.com/energy-modelling-toolkit/prosumpy)
    param_tech_no_batt=param_tech.copy()
    param_tech_no_batt.update({'BatteryCapacity':0,'MaxPower':0})
    
    # We set the battery capacity to zero in some cases to get the direct SC only
    # We iterate through the community and get the dispatching
    return[param_tech,param_tech_no_batt,selection,df_CH,list_names]

In [19]:
def run_comm_as_today_test(pv_penetration,batt_penetration,first_time):
    '''
    Description
    -----------
    runs the core function using selected pv and battery penetration.
    Parameters
    -----------
    pv_penetration (int): % of pv penetration in the community
    batt_penetration (int): % of battery penetration in the community
    first_time (Boolean): select whether it is the first time to generate the community or not (load a pre-defined one)
    Returns
    -----------
    None
    '''
    [param_tech,param_tech_no_batt,selection,df_CH,list_names]=initialize_community(first_time)
    core(pv_penetration,batt_penetration,df_CH,selection,list_names,param_tech,param_tech_no_batt)

In [20]:
def run_comm_as_today(first_time):
    '''
    Description
    -----------
    runs the whole core function.
    Parameters
    -----------
    first_time (Boolean): select whether it is the first time to generate the community or not (load a pre-defined one)    
    Returns
    -----------
    None
    '''
    [param_tech,param_tech_no_batt,selection,df_CH,list_names]=initialize_community(first_time)
    for pv_penetration in param_tech['list_pv_penetration']:
        for batt_penetration in param_tech['list_batt_penetration']:


In [21]:
#%%time
#run_comm_as_today(False)

########################################
PV penetration 100
Battery penetration 100

%%%%%%%%%%%%%%%%%
Printing analysis
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 226949 kWh
Total fed to the grid: 404003 kWh
Total bought from the grid: 35159.3 kWh
Self consumption rate (SCR): 33.1617%
Self sufficiency rate (SSR): 86.586%
Self consumption rate (SCR): 33.2%
Self sufficiency rate (SSR): 86.6%
Amount of energy provided by the battery: 126832 kWh
Average Charging/Discharging depth: 34.7486
Number of equivalent full cycles per year: 12683.2 
Total battery losses: 12574.2 kWh
Total inverter losses: 40289.4 kWh
Total bill: -4747.76 USD/a 
Residue (check): 248.422 kWh
%%%%%%%%%%%%%%%%%
1
%%%%%%%%%%%%%%%%%
Saving analysis
########################################
PV penetration 100
Battery penetration 75

%%%%%%%%%%%%%%%%%
Printing analysis
Total yearly consumption: 262109 kWh
Total PV production: 684371 kWh
Self Consumption: 202428 kWh
Total fed to th